# Un interprete di MiniCaml

La spiegazione sul linguaggio MiniCaml e sullo sviluppo di questo interprete è nel materiale didattico del corso (slides). In queste note saranno sottolineati solo alcuni aspetti implementativi.

## Definizioni di tipi di dato usati nell'interprete

Tipi per la sintassi astratta del linguaggio:

In [30]:
(* Identificatori *)
type ide = string;;

(* I tipi *)
type tname =  TInt | TBool | TString | TPair | TClosure | TBiClosure | TRecClosure | TUnBound | TLimClosure

(* Abstract Expressions = espressioni nella sintassi astratta, 
   compongono l'Albero di Sintassi Astratta *)
type exp = 
    | EInt of int
    | CstTrue 
    | CstFalse
    | EString of string
    | Den of ide
    (* Operatori binari da interi a interi*)
    | Sum of exp * exp
    | Diff of exp * exp
    | Prod of exp * exp
    | Div of exp * exp
    (* Operatori da interi a booleani *)
    | IsZero of exp
    | Eq of exp * exp
    | LessThan of exp*exp
    | GreaterThan of exp*exp
    (* Operatori su booleani *)
    | And of exp*exp
    | Or of exp*exp
    | Not of exp
    (* Controllo del flusso, funzioni *)
    | IfThenElse of exp * exp * exp
    | Let of ide * exp * exp
    | Letrec of ide * ide  * exp * exp
    | Fun of ide * exp
    | Apply of exp * exp
    | Bifun of ide * ide * exp
    | LimFun of ide * exp * exp * exp 
    (* Coppie *)
    | Pair of exp * exp
    | First of exp
    | Second of exp

type ide = string


type tname =
    TInt
  | TBool
  | TString
  | TPair
  | TClosure
  | TBiClosure
  | TRecClosure
  | TUnBound
  | TLimClosure


type exp =
    EInt of int
  | CstTrue
  | CstFalse
  | EString of string
  | Den of ide
  | Sum of exp * exp
  | Diff of exp * exp
  | Prod of exp * exp
  | Div of exp * exp
  | IsZero of exp
  | Eq of exp * exp
  | LessThan of exp * exp
  | GreaterThan of exp * exp
  | And of exp * exp
  | Or of exp * exp
  | Not of exp
  | IfThenElse of exp * exp * exp
  | Let of ide * exp * exp
  | Letrec of ide * ide * exp * exp
  | Fun of ide * exp
  | Apply of exp * exp
  | Bifun of ide * ide * exp
  | LimFun of ide * exp * exp * exp
  | Pair of exp * exp
  | First of exp
  | Second of exp


## Ambiente e valori esprimibili 

Ambiente come tipo polimorfo:

* L'ambiente associa identificatori a valori. L'implementazione più semplice (come nelle slides del corso) è tramite una lista di coppie (identificatore,valore) e una funzione `lookup` fornisce il valore associato ad un dato identificatore
* I questo caso l'ambiente è implementato come una funzione vera e propria "aggiornabile" tramite la funzione `bind` (sotto)
* L'ambiente è definito come tipo polimorfo per consentire la mutua ricorsione con la definizione del tipo dei valori, evT

In [31]:
(* Ambiente polimorfo *)
type 't env = ide -> 't

type 't env = ide -> 't


In [32]:
(* Evaluation types = tipi  esprimibili *)
type evT = 
    | Int of int 
    | Bool of bool 
    | String of string 
    | Closure of ide * exp * evT env
    | BiClosure of ide * ide * exp * evT env
    | RecClosure of ide * ide * exp * evT env
    | LimClosure of ide * int * int * exp * evT env
    | EPair of evT * evT
    | UnBound

type evT =
    Int of int
  | Bool of bool
  | String of string
  | Closure of ide * exp * evT env
  | BiClosure of ide * ide * exp * evT env
  | RecClosure of ide * ide * exp * evT env
  | LimClosure of ide * int * int * exp * evT env
  | EPair of evT * evT
  | UnBound


In [33]:
(* Ambiente vuoto *)
let emptyenv = function x -> UnBound

val emptyenv : 'a -> evT = <fun>


In [34]:
(* Binding fra una stringa x e un valore primitivo evT *)
let bind (s: evT env) (x: ide) (v: evT) = 
    function (i: ide) -> if (i = x) then v else (s i)

val bind : evT env -> ide -> evT -> ide -> evT = <fun>


## Type Checking

In [35]:
(* Funzione da evT a tname che associa a ogni valore il suo descrittore di tipo  *)
let getType (x: evT) : tname =
    match x with
    | Int(n) -> TInt
    | Bool(b) -> TBool
    | String(s) -> TString
    | Closure(i,e,en) -> TClosure
    | BiClosure(i,j,e,en) -> TBiClosure
    | RecClosure(i,j,e,en) -> TRecClosure
    | LimClosure(i,lim0,lim1, e, en) -> TLimClosure
    | EPair(e0, e1) -> TPair
    | UnBound -> TUnBound

val getType : evT -> tname = <fun>


In [36]:
(* Type-checking *)
let typecheck ((x, y) : (tname*evT)) = 
    match x with
    | TInt -> (match y with 
               | Int(u) -> true
               | _ -> false
               )
    | TBool -> (match y with 
                | Bool(u) -> true
                | _ -> false
                )
    | TString -> (match y with
                 | String(u) -> true
                 | _ -> false
                 )
    | TClosure -> (match y with
                   | Closure(i,e,n) -> true
                   | _ -> false
                   )
    | TLimClosure -> (match y with
                   | LimClosure(i,lim1,lim2,e,n) -> true
                   | _ -> false
                   )
    | TBiClosure -> (match y with
                   | BiClosure(i,j,e,n) -> true
                   | _ -> false
                   )
    | TRecClosure -> (match y with
                     | RecClosure(i,j,e,n) -> true
                     | _ -> false
                     )
    | TPair -> (match y with
                     | EPair(e0,e1) -> true
                     | _ -> false
                     )
    |TUnBound -> (match y with
                 | UnBound -> true
                 | _ -> false
                 )

val typecheck : tname * evT -> bool = <fun>


## Eccezione in caso di errori durante l'esecuzione

In [37]:
(* Errori a runtime *)
exception RuntimeError of string

exception RuntimeError of string


## Operazioni primitive

In [38]:
(* PRIMITIVE del linguaggio *)

(* Controlla se un numero è zero *)
let is_zero(x) = match (x) with
    | (Int(v)) -> Bool(v = 0)
    | (_) -> raise ( RuntimeError "Wrong type")

(* Uguaglianza fra interi *)
let int_eq(x,y) =   
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v = w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Somma fra interi *)	   
let int_plus(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v + w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Differenza fra interi *)
let int_sub(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v - w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Prodotto fra interi *)
let int_times(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v * w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")
    
(* Divisione fra interi *)
let int_div(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> 
                    if w<>0 then Int(v / w)
                            else raise (RuntimeError "Division by zero")
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni di confronto *)
let less_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v < w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let greater_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v > w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni logiche *)
let bool_and(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v && w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_or(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v || w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_not(x) = 
    match (typecheck(TBool,x), x) with
    | (true, Bool(v)) -> Bool(not(v))
    | (_,_) -> raise ( RuntimeError "Wrong type")
    
let pair_first(x) =
    match (typecheck(TPair,x), x) with
    | (true, EPair(e0, e1)) -> e0
    | (_,_) -> raise ( RuntimeError "Wrong type")
    
let pair_second(x) =
    match (typecheck(TPair,x), x) with
    | (true, EPair(e0, e1)) -> e1
    | (_,_) -> raise ( RuntimeError "Wrong type")

val is_zero : evT -> evT = <fun>


val int_eq : evT * evT -> evT = <fun>


val int_plus : evT * evT -> evT = <fun>


val int_sub : evT * evT -> evT = <fun>


val int_times : evT * evT -> evT = <fun>


...

val bool_and : evT * evT -> evT = <fun>


val bool_or : evT * evT -> evT = <fun>


val bool_not : evT -> evT = <fun>


val pair_first : evT -> evT = <fun>


val pair_second : evT -> evT = <fun>


## Interprete

In [39]:
(* Interprete *)
let rec eval (e:exp) (s:evT env) : evT = 
    match e with
    | EInt(n) -> Int(n)
    | CstTrue -> Bool(true)
    | CstFalse -> Bool(false)
    | EString(s) -> String(s)
    | Den(i) -> (s i)

    | Prod(e1,e2) -> int_times((eval e1 s), (eval e2 s))
    | Sum(e1, e2) -> int_plus((eval e1 s), (eval e2 s))
    | Diff(e1, e2) -> int_sub((eval e1 s), (eval e2 s))
    | Div(e1, e2) -> int_div((eval e1 s), (eval e2 s))
    
    | Pair(e1, e2) -> EPair(eval e1 s, eval e2 s)
    | First(e) -> pair_first(eval e s)
    | Second(e) -> pair_second(eval e s)

    | IsZero(e1) -> is_zero (eval e1 s)
    | Eq(e1, e2) -> int_eq((eval e1 s), (eval e2 s))
    | LessThan(e1, e2) -> less_than((eval e1 s),(eval e2 s))
    | GreaterThan(e1, e2) -> greater_than((eval e1 s),(eval e2 s))

    | And(e1, e2) -> bool_and((eval e1 s),(eval e2 s))
    | Or(e1, e2) -> bool_or((eval e1 s),(eval e2 s))
    | Not(e1) -> bool_not(eval e1 s)

    | IfThenElse(e1,e2,e3) -> 
        let g = eval e1 s in 
            (match (typecheck(TBool,g),g) with
            |(true, Bool(true)) -> eval e2 s
            |(true, Bool(false)) -> eval e3 s
            |(_,_) -> raise ( RuntimeError "Wrong type")
            )
    | Let(i, e, ebody) -> eval ebody (bind s i (eval e s))
    | Fun(arg, ebody) -> Closure(arg,ebody,s)
    | Bifun(arg0, arg1, ebody) -> BiClosure(arg0,arg1,ebody,s)
    | Letrec(f, arg, fBody, leBody) ->
        let benv = bind (s) (f) (RecClosure(f, arg, fBody,s)) in
            eval leBody benv
    | LimFun(arg, lim1, lim2, body) -> 
        (match (eval lim1 s),(eval lim2 s) with
            | Int l1, Int l2 -> LimClosure(arg, l1, l2, body, s)
            | _ -> raise ( RuntimeError "Wrong type"))
    | Apply(eF, eArg) ->
        let fclosure = eval eF s in 
            (match fclosure with 
            | Closure(arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in 
                let aenv = bind fDecEnv arg aVal in 
                eval fbody aenv 
            | BiClosure(arg0, arg1, fbody, fDecEnv) -> 
                let aVal = eval eArg s in
                let aenv = bind fDecEnv arg0 aVal in
                eval (Fun(arg1, fbody)) aenv
            | LimClosure(arg, lim1, lim2, fbody, fDecEnv) ->
                let aVal = eval eArg s in
                    (match aVal with
                        | Int(x) -> eval (Fun(arg, fbody)) fDecEnv
                        | _ -> raise ( RuntimeError "Wrong type"))
            | RecClosure(f, arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in
                let rEnv = bind fDecEnv f fclosure in
                let aenv = bind rEnv arg aVal in 
                eval fbody aenv
            | _ -> raise ( RuntimeError "Wrong type")
            )

val eval : exp -> evT env -> evT = <fun>


## Esempio di esecuzione: fattoriale

In [42]:
let myRP = IsZero(EInt(0))

val myRP : exp = IsZero (EInt 0)


In [43]:
eval myRP emptyenv;;

- : evT = Bool true
